# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('city_data.csv')
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cayenne,75,GF,1580074704,94,4.93,-52.33,77.00,4.70
1,Andenes,100,NO,1580044204,62,69.31,16.12,21.20,23.04
2,Longyearbyen,0,SJ,0,77,78.22,15.64,1.40,1.12
3,Zhangjiakou,100,CN,1580117391,64,40.81,114.88,17.13,4.38
4,Port Alfred,28,ZA,1580059289,78,-33.59,26.89,64.00,7.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Gmaps
gmaps.configure(api_key = g_key)


In [4]:
locations = city_data[['Lat', 'Lng']].astype(int)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = city_data['Humidity'], dissipating = False, 
                                 max_intensity = 100, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Cloudiness Filter
#0
cloudiness_wish = float(input('What is the max cloudiness you wish?  '))
city_data = city_data.drop(city_data[city_data["Cloudiness"] == cloudiness_wish].index)

What is the max cloudiness you wish?  0


In [6]:
#Max Temp Filter
#80
max_temp_wish = float(input('What is the Max Temp you wish?  '))
city_data = city_data.drop(city_data[city_data["Max Temp"] > max_temp_wish].index)


What is the Max Temp you wish?  80


In [7]:
#Min Temp Filter
#70
min_temp_wish = float(input('What is the Min Temp you wish?  '))
city_data = city_data.drop(city_data[city_data["Max Temp"] <= min_temp_wish].index)


What is the Min Temp you wish?  70


In [8]:
#Wind Speed Filter
#10
wind_speed_wish = float(input('What is the Max Wind Speed you wish?  '))
city_data = city_data.drop(city_data[city_data["Wind Speed"] >= wind_speed_wish].index)
city_data.reset_index(inplace = True)
del city_data['index']
city_data.head()

What is the Max Wind Speed you wish?  10


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cayenne,75,GF,1580074704,94,4.93,-52.33,77.00,4.70
1,Palembang,91,ID,1580124053,94,-2.92,104.75,75.16,1.50
2,Bengkulu,70,ID,1580124720,88,-3.80,102.27,74.55,3.36
3,Kahului,20,US,1580098336,61,20.89,-156.47,78.80,9.17
4,Valparaiso,2,CL,1580082857,53,-33.04,-71.63,77.00,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
city_data["Hotel"] = " "
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    "rankby": "distance",
    "name": "hotel",
    "key": g_key,}

for i in range(0, len(city_data)):
    lat = city_data.loc[i, "Lat"]
    lng = city_data.loc[i, "Lng"]
    params["location"] = f'{lat},{lng}'
    hotel = requests.get(base_url, params=params)
    hotel = hotel.json()
    try:
        city_data.loc[i, "Hotel"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result...... Skipping.")
city_data.head()

Missing field/result...... Skipping.
Missing field/result...... Skipping.
Missing field/result...... Skipping.
Missing field/result...... Skipping.
Missing field/result...... Skipping.
Missing field/result...... Skipping.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel
0,Cayenne,75,GF,1580074704,94,4.93,-52.33,77.00,4.70,Hotel Central,Hotel Central
1,Palembang,91,ID,1580124053,94,-2.92,104.75,75.16,1.50,OYO 1299 Max To U Hotel,OYO 1299 Max To U Hotel
2,Bengkulu,70,ID,1580124720,88,-3.80,102.27,74.55,3.36,Hotel Santika Bengkulu,Hotel Santika Bengkulu
3,Kahului,20,US,1580098336,61,20.89,-156.47,78.80,9.17,Maui Seaside Hotel,Maui Seaside Hotel
4,Valparaiso,2,CL,1580082857,53,-33.04,-71.63,77.00,8.05,Hotel Casa Higueras,Hotel Casa Higueras


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_data.iterrows()]
locations = city_data[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = city_data["Hotel"], dissipating = False, 
                                 max_intensity = 100, point_radius = 1)
fig.add_layer(heat_layer)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))